# The Iterated Prisoner's Dilemma

Author : Shayan Firouzian H.

## Introduction

The Prisoner's Dilemma (PD) is a **simultaneous**, **two-player**, **non-zero-sum** game, highlighted by Merill Flood & Melvin Dreschler in 1950 to show that Nash's balance is not always ideal. The iterated version of the game (IPD) allows you to express strategies that are based on the game history, and therefore learn from the past. In 1980 Robert Axelrod organized a competition for the iterated version of the game in which one of the participants, Anatol Rappoport, highlighted the famous TFT strategy. This iterated version and TFT strategy were popularized in 1984 in Robert Axelrod's book "The Evolution of Cooperation".

In [ ]:
%run ../src/game.py

dip =[(3,3),(0,5),(5,0),(1,1)]   # Prisoner's dilemma
g = Game(dip,['C','D'])
g.getNash()


# Strategy

#### Create a class of strategies

In [ ]:
from abc import abstractmethod


class Strategy:
    def setMemory(self, mem):
        pass

    def getAction(self, tick):
        pass

    def __copy__(self):
        pass

    def update(self, x, y):
        pass


class Periodic(Strategy):
    def __init__(self, sequence, name=None):
        super().__init__()
        self.sequence = sequence.upper()
        self.step = 0
        self.name = "per_" + sequence if (name == None) else name

    def getAction(self, tick):
        return self.sequence[tick % len(self.sequence)]

    def clone(self):
        object = Periodic(self.sequence, self.name)
        return object

    def update(self, x, y):
        pass


print("All is ok")

# One meeting

In [ ]:
class Meeting:
    def __init__(self, game, s1, s2, length=1000):
        self.game = game
        self.s1 = s1.clone()
        self.s2 = s2.clone()
        self.length = length
        self.nb_cooperation_s1 = 0
        self.nb_cooperation_s2 = 0

    def reinit(self):
        self.s1_score = 0
        self.s2_score = 0
        self.s1_rounds = []
        self.s2_rounds = []

    def run(self):
        self.reinit()
        for tick in range(0, self.length):
            c1 = self.s1.getAction(tick).upper()
            c2 = self.s2.getAction(tick).upper()
            if c1 == "C":
                self.nb_cooperation_s1 += 1
            if c2 == "C":
                self.nb_cooperation_s2 += 1
            self.s1_rounds.append(c1)
            self.s2_rounds.append(c2)
            self.s1.update(c1, c2)
            self.s2.update(c2, c1)
            act = self.game.actions
            self.s1_score += self.game.scores["x"][act.index(c1), act.index(c2)]
            self.s2_score += self.game.scores["y"][act.index(c1), act.index(c2)]

    def prettyPrint(self,max=20) :
        print("{:8}\t{} = {}".format(self.s1.name, ' '.join(map(str, self.s1_rounds)) , self.s1_score))
        print("{:8}\t{} = {}".format(self.s2.name, ' '.join(map(str, self.s2_rounds)) , self.s2_score))

print("All is ok")

In [ ]:
dip =[(3,3),(0,5),(5,0),(1,1)]   # Prisoner's dilemma
g = Game(dip,['C','D'])
s1=Periodic("CCD")
s2=Periodic("DDC")
m = Meeting(g,s1,s2,10)
m.run()
m.prettyPrint()
# We must get 15,35
print()
print("Number of cooperations : " )
print (m.s1.name+"\t" + str(m.nb_cooperation_s1))
print (m.s2.name+"\t" + str(m.nb_cooperation_s2))

# Run a tournament

In [ ]:
import pandas as pd
import numpy as np


class Tournament:
    def __init__(self, game, strategies, length=1000, repeat=1):
        self.strategies = strategies
        self.game = game
        self.length = length
        self.repeat = repeat
        size = len(strategies)
        df = pd.DataFrame(np.zeros((size, size + 1), dtype=np.int32))
        df.columns, df.index = (
            [s.name for s in self.strategies] + ["Total"],
            [s.name for s in self.strategies],
        )
        self.matrix = df
        df2 = pd.DataFrame(np.zeros((size, size + 1), dtype=np.int32))
        df2.columns, df2.index = (
            [s.name for s in self.strategies] + ["Total"],
            [s.name for s in self.strategies],
        )
        self.cooperations = df2

    def run(self):
        for k in range(self.repeat):
            for i in range(0, len(self.strategies)):
                for j in range(i, len(self.strategies)):
                    meet = Meeting(
                        self.game, self.strategies[i], self.strategies[j], self.length
                    )
                    meet.run()
                    self.matrix.at[
                        self.strategies[i].name, self.strategies[j].name
                    ] += meet.s1_score
                    if (i != j):
                        self.matrix.at[
                            self.strategies[j].name, self.strategies[i].name
                        ] += meet.s2_score
                    self.cooperations.at[
                        self.strategies[i].name, self.strategies[j].name
                    ] += meet.nb_cooperation_s1
                    if (i != j):
                        self.cooperations.at[
                            self.strategies[j].name, self.strategies[i].name
                        ] += meet.nb_cooperation_s2
        self.matrix["Total"] = self.matrix.sum(axis=1)
        self.matrix.sort_values(by="Total", ascending=False, inplace=True)
        rows = list(self.matrix.index) + ["Total"]
        self.matrix = self.matrix.reindex(columns=rows)
        self.cooperations["Total"] = self.cooperations.sum(axis=1)
        self.cooperations.sort_values(by="Total", ascending=False, inplace=True)
        rows = list(self.cooperations.index) + ["Total"]
        self.cooperations = self.cooperations.reindex(columns=rows)


print("All is ok")

In [ ]:
bag = []
bag.append(Periodic('C'))
bag.append(Periodic('D'))
bag.append(Periodic('DDC'))
bag.append(Periodic('CCD'))
t=Tournament(g,bag,10)
t.run()
print("The score matrix: ")
print(t.matrix)
print()
# SUR 10 COUPS : [('per_D', 120), ('per_DDC', 102), ('per_CCD', 78), ('per_C', 60)]
print("The cooperation matrix: ")
print(t.cooperations)

## Generate sets of strategies

In [ ]:
import itertools
cards = ['C','D']
periodics = [p for p in itertools.product(cards, repeat=1)]+[p for p in itertools.product(cards, repeat=2)] + [p for p in itertools.product(cards, repeat=3)]
strats = [Periodic(''.join(p)) for p in periodics] # join to transform in strings
print(str(len(strats))+" stratégies générées")
# 14 are generated: 2 with period lengh of 1, 4 with period lengh of 2, 8 with period lengh of 3

# Ecological competitions

In [ ]:
import pandas
import copy
import math
import matplotlib.pyplot as plt

%matplotlib inline


class Ecological:
    def __init__(self, game, strategies, length=1000, repeat=1, pop=100):
        self.strategies = strategies
        self.pop = pop
        self.game = game
        self.length = length
        self.generation = 0  # Number of the current generation
        self.base = pop * len(strategies)
        self.historic = pandas.DataFrame(columns=[strat.name for strat in strategies])
        self.historic.loc[0] = [pop for x in range(len(strategies))]
        self.extinctions = dict((s.name, math.inf) for s in strategies)
        self.cooperations = dict((s.name, 0) for s in strategies)
        self.listeCooperations = list()
        self.scores = dict((s.name, 0) for s in strategies)
        self.tournament = Tournament(self.game, self.strategies, length, repeat)
        self.tournament.run()

    def run(self):
        dead = 0
        stab = False
        while (self.generation < 1000) and (stab == False):
            parents = list(copy.copy(self.historic.loc[self.generation]))
            for i in range(len(self.strategies)):
                strat = self.strategies[i].name
                if self.historic.at[self.generation, strat] != 0:
                    score = 0
                    cooperations = 0
                    for j in range(len(self.strategies)):
                        strat2 = self.strategies[j].name
                        if self.historic.at[self.generation, strat2] != 0:
                            if i == j:
                                score += (
                                    self.historic.at[self.generation, strat] - 1
                                ) * self.tournament.matrix.at[strat, strat2]
                                cooperations += (
                                    self.historic.at[self.generation, strat] - 1
                                ) * self.tournament.cooperations.at[strat, strat2]
                            else:
                                score += (
                                    self.historic.at[self.generation, strat2]
                                    * self.tournament.matrix.at[strat, strat2]
                                )
                                cooperations += (
                                    self.historic.at[self.generation, strat2]
                                    * self.tournament.cooperations.at[strat, strat2]
                                )
                        self.scores[strat] = score
                        self.cooperations[strat] = cooperations

            total = 0
            totalCooperations = 0
            for strat in self.strategies:
                total += (
                    self.scores[strat.name]
                    * self.historic.at[self.generation, strat.name]
                )
                totalCooperations += (
                    self.cooperations[strat.name]
                    * self.historic.at[self.generation, strat.name]
                )
            for strat in self.strategies:
                parent = self.historic.at[self.generation, strat.name]
                if self.scores[strat.name] != 0:
                    self.historic.at[self.generation + 1, strat.name] = math.floor(
                        self.base * parent * self.scores[strat.name] / total
                    )
                elif self.scores[strat.name] == 0:
                    self.historic.at[self.generation + 1, strat.name] = 0
                    dead += 1
                if (parent != 0) and (
                    self.historic.at[self.generation + 1, strat.name] == 0
                ):
                    self.extinctions[strat.name] = self.generation + 1
                elif self.historic.at[self.generation + 1, strat.name] != 0:
                    self.extinctions[strat.name] = (
                        self.historic.at[self.generation + 1, strat.name] * 1000
                    )
                if dead == len(self.strategies) - 1:
                    stab = True
            self.listeCooperations.append(
                totalCooperations / (self.base * self.length * len(self.strategies))
            )
            self.generation += 1
            if parents == list(self.historic.loc[self.generation]):
                stab = True
        trie = sorted(self.extinctions.items(), key=lambda t: t[1], reverse=True)
        df_trie = pandas.DataFrame()
        for t in trie:
            df_trie[t[0]] = self.historic[t[0]]
        self.historic = df_trie
        return self.historic

    def saveData(self):
        date = datetime.datetime.now()
        self.historic.to_csv(str(date) + ".csv", sep=";", encoding="utf-8")

    def drawPlot(self, nbCourbes=None, nbLegends=None):
        nbCourbes = len(self.strategies) if (nbCourbes == None) else nbCourbes
        nbLegends = len(self.strategies) if (nbLegends == None) else nbLegends
        strat = self.historic.columns.tolist()
        for i in range(nbCourbes):
            plt.plot(
                self.historic[strat[i]],
                label=strat[i] if (i < nbLegends) else "_nolegend_",
            )
        plt.legend(bbox_to_anchor=(0, 1), loc=2, borderaxespad=0.0)
        plt.ylabel("Population")
        plt.xlabel("Generation")
        plt.show()
        # date = datetime.datetime.now()
        # plt.savefig(str(date)+'.png', dpi=1000)

    def drawCooperation(self):
        plt.plot(self.listeCooperations)
        plt.ylabel("Percentage of cooperations")
        plt.xlabel("Generation")
        plt.ylim(0, 101)
        plt.show()


print("All is ok")

#### Organize an ecological competition with All_C (which always cooperates) and All_D (which always betrays)
Once the competition is completed, it is possible to display the population evolution curve of each strategy as well as the evolution curve of the cooperations played in each generation.

In [ ]:
All_C = Periodic("C","All_C")
All_D = Periodic("D","All_D")
eco = Ecological(g, [All_C, All_D])
eco.run()
print("Evolution de la population")
eco.drawPlot()
print("Historique de la population")
print(eco.historic)
print("Evolution des cooperations")
eco.drawCooperation()
print(eco.scores)

# Reactive strategies
Strategies are called "reactive" if their actions depend on the opponent's past actions. Some of them are very simple to understand. Among the most famous are
- `Tft` (abbreviation of "tit for tat" or "donnant-donnant" as we would say in French) which starts by cooperating and then plays the same thing as the opponent on the previous round
- `Spiteful` who cooperates as long as the opponent has cooperated, but who never forgives him if he has betrayed once 

In [ ]:
class Tft(Strategy):
    def __init__(self):
        super().__init__()
        self.name = "tft"
        self.hisPast = ""

    def getAction(self, tick):
        return "C" if (tick == 0) else self.hisPast[-1]

    def clone(self):
        return Tft()

    def update(self, my, his):
        self.hisPast += his


class Spiteful(Strategy):
    def __init__(self):
        super().__init__()
        self.name = "spiteful"
        self.hisPast = ""
        self.myPast = ""

    def getAction(self, tick):
        if tick == 0:
            return "C"
        if self.hisPast[-1] == "D" or self.myPast[-1] == "D":
            return "D"
        else:
            return "C"

    def clone(self):
        return Spiteful()

    def update(self, my, his):
        self.myPast += my
        self.hisPast += his


print("All is ok")

#### Behaviour of these reactive strategies
Let's check the behavior of these two new strategies against `Periodic("CCD")` in a Meeting.

In [ ]:
m = Meeting(g,Tft(),Periodic("CCD"),10)
m.run()
m.prettyPrint()
print("")
m = Meeting(g,Spiteful(),Periodic("CCD"),10)
m.run()
m.prettyPrint()

## Generate them all
For a `Mem(x,y)` family, the genome is of size `max(x,y)` for the first rounds plus `2^(x+y)` for all situations `s` of the past on `x` moves of one player and `y` moves of the other. So there are `2^(max(x,y)+2^(x+y))` strategies to generate. To obtain all these elements, it is therefore sufficient to compute all the possible instanciations of C and D in the genome, which is done, once again, with a Cartesian product.


| family  | genome length | number of strats  |
|         :-:   |     :-:     | :-:    |
| mem(0,1) | 1+2^1 = 3        | 2^3 = 8 |
| mem(1,0) | 1+2^1 = 3        | 2^3 = 8 |
| mem(1,1) | 1+2^2 = 5        | 2^5 = 32 |
| mem(2,0) | 2+2^2 = 6        | 2^6 = 64 |
| mem(1,2) | 2+2^3 = 10       | 2^10 = 1024 |
| mem(2,1) | 2+2^3 = 10       | 2^10 = 1024 |
| mem(2,2) | 2+2^4 = 18       | 2^18 = 262144 |


In [ ]:
def getMem(x,y):
    if (x+y > 4):
        return "Pas calculable"
    len_genome = max(x,y)+2**(x+y)
    permut = [p for p in itertools.product(['C','D'], repeat=len_genome)]
    genomes = [''.join(p) for p in permut]
    return [Mem(x,y,gen) for gen in genomes]


print("In Mem(1,1) there are "+ str(len(getMem(1,1))) + " strategies")

## The Mem(1,1) competition